In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers
import games_configurations
env_name = 'BipedalWalkerHardcore-v2'#'RoboschoolHalfCheetah-v1'#'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
#a2c_config = games_configurations.pendulum_lstm_config
#a2c_config = games_configurations.halfcheetah_lstm_config_v2
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
#a2c_config = games_configurations.bipedalwalker_config
#a2c_config = games_configurations.roboschoolhumanoid_lstm_config
a2c_config = games_configurations.bipedalwalkerhardcore_lstm_config
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(obs_space)
print(action_space)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Box(48,)
Box(4,)


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-07-25 10:02:27,892	WARNING worker.py:1337 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-07-25 10:02:27,892	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-25_10-02-27_5921/logs.
2019-07-25 10:02:28,009	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:42682 to respond...


2019-07-25 10:02:28,130	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:41474 to respond...
2019-07-25 10:02:28,131	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-07-25 10:02:28,154	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-25_10-02-27_5921/logs.
2019-07-25 10:02:28,168	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.
2019-07-25 10:02:28,183	WARNING services.py:907 -- Failed to start the reporter. The reporter requires 'pip install psutil'.


{'node_ip_address': '192.168.3.115',
 'redis_address': '192.168.3.115:42682',
 'object_store_address': '/tmp/ray/session_2019-07-25_10-02-27_5921/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-07-25_10-02-27_5921/sockets/raylet',
 'webui_url': None}

In [ ]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks



agent = A2CAgent(sess,'run2', obs_space, False, action_space, a2c_config)
agent.restore('nn/latest_run2')
agent.train()
#agent.save('nn/latest')

W0725 10:02:28.457378 139620787287872 deprecation_wrapper.py:119] From /home/trrrrr/Documents/github/ml/dqn_atari/a2c_continuous.py:81: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 10:02:28.506402 139620787287872 deprecation_wrapper.py:119] From /home/trrrrr/Documents/github/ml/dqn_atari/networks.py:403: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0725 10:02:28.514101 139620787287872 deprecation.py:323] From /home/trrrrr/Documents/github/ml/dqn_atari/networks.py:409: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0725 10:02:28.526521 139620787287872 deprecation.py:506] From /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed

0
1
2
3
4
5
6
7
(pid=6101) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=6101)   result = entry_point.load(False)
(pid=6101) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6101) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6113) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=6113)   result = entry_point.load(False)


W0725 10:02:31.076210 139620787287872 deprecation_wrapper.py:119] From /home/trrrrr/Documents/github/ml/dqn_atari/models.py:96: The name tf.log is deprecated. Please use tf.math.log instead.

W0725 10:02:31.271361 139620787287872 deprecation_wrapper.py:119] From /home/trrrrr/Documents/github/ml/dqn_atari/a2c_continuous.py:164: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0725 10:02:31.272653 139620787287872 deprecation_wrapper.py:119] From /home/trrrrr/Documents/github/ml/dqn_atari/a2c_continuous.py:165: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.



(pid=6109) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=6109)   result = entry_point.load(False)
(pid=6113) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6113) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6109) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6109) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
0
(pid=6121) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6121) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6121) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registr

W0725 10:02:31.303670 139620787287872 deprecation.py:323] From /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(pid=6116) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6116) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6123) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=6123)   result = entry_point.load(False)
(pid=6103) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6103) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6123) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6123) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=6102) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registrat

W0725 10:02:32.871976 139620787287872 deprecation_wrapper.py:119] From /home/trrrrr/Documents/github/ml/dqn_atari/a2c_continuous.py:172: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

W0725 10:02:33.308661 139620787287872 deprecation.py:323] From /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Frames per seconds:  2194.8664243692842
saving next best rewards:  11.575313889799016
Frames per seconds:  3380.898548655922
saving next best rewards:  20.874123146903127
Frames per seconds:  3885.311687595042
saving next best rewards:  23.554213686140738
Frames per seconds:  3747.218832266403
saving next best rewards:  23.77867223799703
Frames per seconds:  3792.237901830177
saving next best rewards:  27.1355824841495
Frames per seconds:  3718.9985522205056
saving next best rewards:  29.360637731942223
Frames per seconds:  3694.629423983245
Frames per seconds:  3811.5948171381965
saving next best rewards:  31.464500796076337
Frames per seconds:  3697.4962689008494
saving next best rewards:  31.4988989369111
Frames per seconds:  3727.2747723915168
saving next best rewards:  31.563042195701073
Frames per seconds:  3628.6411340934624
saving next best rewards:  32.70215515738986
Frames per seconds:  3613.836256771963
saving next best rewards:  33.58649280786461
Frames per seconds:  3704.3

Frames per seconds:  3212.7334144023807
Frames per seconds:  3318.339156159371
Frames per seconds:  3652.8633274393787
Frames per seconds:  3669.964928710709
Frames per seconds:  3323.243223279025
Frames per seconds:  3590.761511425313
Frames per seconds:  3753.199240907125
Frames per seconds:  3640.141100119192
Frames per seconds:  3279.4095765597167
Frames per seconds:  3300.2000850861323
Frames per seconds:  3311.9633313900945
Frames per seconds:  3404.604267030814
Frames per seconds:  3404.590772999923
Frames per seconds:  3362.888846085787
Frames per seconds:  3457.292887662482
Frames per seconds:  3479.8280214576175
Frames per seconds:  3826.502611752694
Frames per seconds:  3648.892038982981
Frames per seconds:  3583.2232925470335
Frames per seconds:  3637.9776383373764
Frames per seconds:  3621.9997436329904
Frames per seconds:  3574.796036149412
Frames per seconds:  3434.620138516073
Frames per seconds:  3661.323931175111
Frames per seconds:  3471.098577525698
Frames per secon

Frames per seconds:  3682.8905317122585
Frames per seconds:  3570.5492668887437
Frames per seconds:  3582.8825303323833
Frames per seconds:  3743.3780454217426
Frames per seconds:  3718.0284309425424
Frames per seconds:  3693.9716526728535
Frames per seconds:  3523.144897102058
Frames per seconds:  3687.0818057652837
Frames per seconds:  3731.856302888279
Frames per seconds:  3690.993960015503
Frames per seconds:  3787.647292539116
Frames per seconds:  3946.397766845596
Frames per seconds:  3791.6698829649586
Frames per seconds:  3630.869213883355
Frames per seconds:  3776.3199600471976
Frames per seconds:  3680.765344494836
Frames per seconds:  3601.330612493596
Frames per seconds:  3787.0338989875595
Frames per seconds:  3791.9153712206
Frames per seconds:  3674.793261145033
Frames per seconds:  3326.8796853266836
Frames per seconds:  3598.0401179641167
Frames per seconds:  3675.7975736983767
Frames per seconds:  3633.2916062288373
Frames per seconds:  3587.842922288986
Frames per se

In [ ]:
agent.save('nn/latest_run2')
ray.shutdown()

In [ ]:
import gym
gym.envs.registry.all()